# 이름/학번

이름: 나영민

학번: 20183217

In [2]:
import numpy as np
import torch

# Example dataset

강의를 위해서 임의의 dataset을 준비하겠습니다.
예제로 봐주시고, 큰 물리적 의미는 부여하지 않겠습니다.

- Data는 장미과와 국화과의 A 효소, B 효소, C 효소, D 효소를 측정한 값이라고 가정합니다.
- Label은 각 sample이 장미인지 (0) 국화인지 (1)에 대한 정보라고 가정합니다.

In [3]:
batch_size = 10
num_feature = 4
torch.manual_seed(0)

X_batch = torch.randn(batch_size, num_feature)
Y_batch = (torch.sum(X_batch, dim=1)>0).type(torch.float).reshape(-1,1)

In [4]:
X_batch

tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        [ 0.1198,  1.2377,  1.1168, -0.2473],
        [-1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530],
        [-0.2159, -0.7425,  0.5627,  0.2596],
        [-0.1740, -0.6787,  0.9383,  0.4889],
        [ 1.2032,  0.0845, -1.2001, -0.0048],
        [-0.5181, -0.3067, -1.5810,  1.7066]])

In [5]:
Y_batch
# 첫번째 꽃: 장미
# 두번째 꽃: 장미

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.]])

# Notation 정리

강의자료와 비교하면 

\begin{align*}
\text{X_batch} = 
\begin{bmatrix}
(x^{(1)})^\top\\
(x^{(2)})^\top\\
\vdots \\
(x^{(10)})^\top
\end{bmatrix}, \quad
\text{Y_batch} = 
\begin{bmatrix}
y^{(1)}\\
y^{(2)}\\
\vdots \\
y^{(10)}
\end{bmatrix}
\end{align*}

# Example Problem: Single Neuron

- 한개의 neuron이 있다고 가정하고 $\mathbb{R}^4$ 를 입력받아서 $\mathbb{R}$로 출력한다고 가정합니다.
- Activation 함수는 ReLU 함수, 즉, 
\begin{align*}
\text{ReLU}(x) = \max(0, x)
\end{align*}
를 사용한다고 가정합니다.

Neuron을 통해서 batch sample 전체를
\begin{align*}
Z = \begin{bmatrix}
(w^T x^{(1)} + b)^T \\
(w^T x^{(2)} + b)^T \\
\vdots \\
(w^T x^{(m)} + b)^T
\end{bmatrix} 
\end{align*}
연산을 수행해서 $Z$를 구하세요.

- $w$는 random Gaussian으로 생성하세요. 위에서 예기한 입력과 출력이 맞도록 weight를 생성하세요.
- Bias $b$는 1로 설정




In [6]:
# 답 작성

W = torch.randn(num_feature, 1)
b = 1
Z = torch.matmul(X_batch, W)+b
print(Z)

tensor([[ 2.2910],
        [ 3.0975],
        [ 1.6780],
        [ 1.0120],
        [ 1.4991],
        [-0.5044],
        [ 1.4891],
        [ 1.3363],
        [ 0.2980],
        [-1.8288]])


원하는 연산을 하는지 확인하도록 합니다

In [7]:
z_loop = torch.empty(batch_size, 1)
for i in torch.arange(batch_size):
    z_loop[i,:] = torch.matmul(W.T, X_batch[i,:].T)+b
print(z_loop)

tensor([[ 2.2910],
        [ 3.0975],
        [ 1.6780],
        [ 1.0120],
        [ 1.4991],
        [-0.5044],
        [ 1.4891],
        [ 1.3363],
        [ 0.2980],
        [-1.8288]])


/var/folders/0q/f448gzfd2rd9vk0q5h6wlvn80000gn/T/ipykernel_1669/1446079948.py:3: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3281.)
  z_loop[i,:] = torch.matmul(W.T, X_batch[i,:].T)+b


## Problem 1
ReLU 함수를 작성하고 위에서 찾은 $Z$의 각 원소에 ReLU 함수를 적용하여 `a`라는 변수에 저장하세요.

- torch.clamp() 함수를 공부하고 적용하세요




In [8]:
# 답 작성:

def ReLU(x):
    return torch.clamp(x,min=0)

A = ReLU(Z)

print(A)

tensor([[2.2910],
        [3.0975],
        [1.6780],
        [1.0120],
        [1.4991],
        [0.0000],
        [1.4891],
        [1.3363],
        [0.2980],
        [0.0000]])


# Single Layer Network

- 한개의 Layer에 $k=5$개의 Neuron 이 있는 network를 구성하고 출력을 구하세요
- Activation function은 모든 neuron에 ReLU를 적용합니다
- 모든 weight는 Gaussian 분포로 랜덤 생성하세요 `torch.randn()`
- $i$ 번째 neuron의 weight들을 $w_i$라고 할때,
\begin{align*}
\text{W} = \begin{bmatrix}
w_1, w_2, w_3, w_4, w_5
\end{bmatrix}
\end{align*}
라고 하고, weight matrix `W`를 만드세요.
  - `W = torch.randn(4, 5)` 으로 생성
- Bias 역시 `b`라는 `tensor`에 저장하고, 각 neuron 별로 `1`로 설정합니다
  - `b = torch.ones(1,5)`
- 아래 problem 2-2에서 수업에서 배운 $Z$ 행렬과 $A$ 행렬을 구하세요

# Problem 2-1
`Z` 행렬과 `A` 행렬의 차원은 어떻게 나와야하나요?

답 작성)

Z 는 (10,5), 
A 는 (10,5)

# Problem 2-2 
위에서 요구한 코딩을 완성하세요

In [9]:
# 답 작성
k = 5

W = torch.randn(num_feature,k) 
b = torch.ones(1,5) 
Z = torch.matmul(X_batch,W) + b 
A = ReLU(Z)

print(A)
print(A.shape)

tensor([[1.5009, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 5.6229, 3.3937, 0.3508, 0.0000],
        [1.9497, 0.0000, 0.4126, 1.3829, 2.4589],
        [1.5662, 3.2170, 0.5114, 0.0000, 0.0751],
        [3.6741, 0.0000, 0.0000, 0.0000, 2.0759],
        [2.1981, 0.0000, 0.7003, 3.8946, 5.3346],
        [2.2619, 0.0000, 0.0000, 0.4367, 1.6558],
        [2.8142, 0.0000, 0.0000, 0.3528, 2.0421],
        [0.0000, 1.7287, 3.6229, 3.6950, 1.9198],
        [0.7818, 0.0000, 1.2625, 3.4941, 3.4477]])
torch.Size([10, 5])


# Problem 3: 

- $x^{(3)}$ 를 입력으로하는 2번째 Neuron의 결과값을 출력하세요
- 위에서 구한 `A[i, j]`  인덱싱을 통해서 출력하세요
- `Python`의 인덱싱은 `0`부터 시작한다는 것을 주의하세요

In [13]:
# 답 작성

print('answer = ', A[2, 1])

answer =  tensor(1.2113)


# Problem 4: Multi-Layer network

- 3개의 layer가 있는 network를 구성합니다.
- 2번째 layer의 입력 크기는 $k^{[1]}=16$, 출력 크기는 $k^{[2]}=6$
- 마지막 layer의 출력은 $k^{[3]}=1$개의 neuron으로 구성
- 각 layer의 연산값을 구하세요. 
  - 각 layer의 선형 변환 결과값은 `Z1`, `Z2`, `Z3`로 저장하세요
  - 각 layer의 결과값은 `A1`, `A2`, `A3`로 저장하세요
- 모든 weight는 Gaussian 랜덤 변수로 생성, bias는 1로 구성된 벡터로 생성함
- 각 layer의 weight는 `W1`, `W2`, `W3`로하고, bias는 `b1`, `b2`, `b3`로 생성함
- Activation 함수는 `ReLU`를 적용하세요
- Loop 없이 행렬연산으로 구생하세요

In [11]:
# 답 작성

W1= torch.randn(num_feature,16)
W2= torch.randn(16,6)
W3= torch.randn(6,1)
b1= torch.ones(1,16)
b2= torch.ones(1,6)
b3= torch.ones(1,1)

Z1 = torch.matmul(X_batch,W1) + b1
A1 = ReLU(Z1)
Z2 = torch.matmul(A1, W2) + b2
A2 = ReLU(Z2)
Z3 = torch.matmul(A2, W3) + b3
A3 = ReLU(Z3)





In [12]:
print(Z1.shape)
print(Z1)


torch.Size([10, 16])
tensor([[ 0.8317, -0.6308,  0.5051,  0.8553,  2.5444,  1.3594,  1.1264,  2.9787,
          2.8035, -1.9413,  1.4831, -0.1551,  0.2352,  3.0498, -1.7288,  3.3823],
        [ 1.6584,  2.3320,  0.6552, -1.9099, -4.9303, -1.0179, -1.5007,  1.1670,
          3.4239, -0.3064,  0.7731,  1.0384,  0.5742,  5.0414, -0.4867,  2.3868],
        [-0.0393,  0.1352,  1.3750,  2.6334,  1.0723,  2.3457,  1.0198,  0.8499,
          1.2183,  0.9049,  1.5732, -1.0158, -0.6848,  0.9812,  0.3596,  0.7113],
        [ 0.9995,  1.8367,  0.1942,  0.7268,  0.2951, -1.5948,  0.6320, -0.7372,
         -1.0405,  2.3720, -1.1693,  0.9175,  4.3993,  0.7099,  1.5059,  0.3437],
        [-0.0869, -1.4654,  0.5757,  3.2178,  5.5183,  2.1322,  2.3527,  2.0772,
          0.7470, -0.6851,  1.0313, -1.2825,  0.9458,  0.6877, -1.0220,  2.1722],
        [-0.0358,  0.1650,  2.5580,  4.1172,  3.4414,  5.0769,  2.4867,  0.7870,
          0.3578,  2.6434,  3.0948,  0.3180, -2.5209, -2.3563,  2.8855, -1.1864],
 

In [13]:
print(Z2.shape)
print(Z2)


torch.Size([10, 6])
tensor([[ 1.2972e+01,  1.5879e+01,  1.3625e+01,  6.1362e+00, -3.9340e-01,
         -1.3898e+00],
        [ 1.4109e+01,  2.2289e+01,  1.6362e+01,  4.8086e+00, -4.1622e+00,
         -4.8496e+00],
        [ 7.2885e+00,  8.5322e+00,  1.1982e+01,  9.8866e+00,  1.9092e+00,
          6.4297e+00],
        [ 1.1031e+01,  3.6901e+00,  1.4728e+01,  1.4075e+01, -3.7502e+00,
         -6.3505e+00],
        [ 1.3316e+01,  7.3238e+00,  1.2910e+01,  5.3541e+00, -2.0667e+00,
          2.6020e+00],
        [ 7.7682e+00,  7.2902e+00,  2.2498e+01,  1.4998e+01,  4.8421e-01,
          1.1210e+01],
        [ 1.0140e+01,  7.1445e+00,  1.0190e+01,  8.4843e+00,  1.3260e-01,
          3.1225e+00],
        [ 1.0944e+01,  4.4197e+00,  1.0073e+01,  1.0921e+01, -3.3741e-03,
          3.0654e+00],
        [ 1.2239e+00,  7.9556e+00,  2.5552e+01,  5.8155e+00,  7.4608e-01,
         -3.4636e+00],
        [ 6.9859e+00,  1.4315e+00,  2.7884e+01,  4.5464e+00,  7.8779e-01,
          6.4422e-01]])


In [14]:
print(Z3.shape)
print(Z3)

torch.Size([10, 1])
tensor([[15.7598],
        [26.9483],
        [ 8.5776],
        [-4.0453],
        [12.3496],
        [20.5565],
        [ 2.6170],
        [-4.9203],
        [45.3721],
        [45.5209]])


# Problem 5
- 위에서 구한 `A3[i,j]`의 인덱싱을 통해서 $h_\theta(x^{(3)})$ 을 출력하세요
- 역시 `python`인덱싱은 `0` 부터 시작한다는 것을 주의하세요

In [15]:
# 답 작성

print('h_theta(x3) = ', A3[2,:])

h_theta(x3) =  tensor([8.5776])


# Problem 6
위에서 공부한 것을 함수로 만들어 보도록 하겠습니다.
아래 한 layer의 선형 변환을 연산하는 class를 만들어 보도록 하죠.

- Class는 `my_linear_layer()`
  - `__init__(self, n_input, n_output)` 함수:
    - `self.W` 변수 초기화: Weight 행렬 `self.W`를 램덤 Gaussian 생성 (차원에 맞는...)
    - `self.b` 변수 초기화: bias 벡터 `self.b`를 모두 `1`인 벡터 생성 (차원에 맞는...)
  - `forward(A)` 함수:
    - 입력: `A`는 sample batch $X$ 또는 전 layer에서 들어오는 입력 batch $A^{[\ell-1]}$을 입력하는 자리
    - return 값
      - `Z` 변수는 $A^{[\ell-1]}$의 선형 변환 값, 즉 $Z^{[\ell]}$



In [16]:
# 답 작성
class my_linear_layer():
    def __init__(self, n_input, n_output):
        self.W = torch.randn(n_input, n_output)
        self.b = torch.ones(1,n_output)
    
    def forward(self,A):
        Z = torch.matmul(A, self.W) + self.b
        return Z
    
    
    

답을 확인하기 위해서 `n_input=num_feature`과 `n_output = 5` 인 `my_linear_layer` instance 생성

In [18]:
torch.manual_seed(0)

mll = my_linear_layer(num_feature, 5)
mll.forward(X_batch)

tensor([[ 4.0638,  2.2198, -1.7476, -0.3184,  0.2471],
        [ 0.2967, -2.2693, -1.0325, -3.7866,  0.9864],
        [ 2.8471,  1.4401, -0.5475,  1.4418,  1.7400],
        [ 0.7613,  0.5061,  4.5036,  2.8031,  1.9201],
        [ 5.3088,  4.0278, -0.0173,  3.2829,  1.0363],
        [ 0.6880,  2.4766, -0.1730,  3.1860,  1.3804],
        [ 2.9911,  1.8939,  0.7868,  2.1593,  1.5343],
        [ 3.1934,  2.1373,  1.7049,  3.2263,  1.9170],
        [-2.2147, -0.7392, -0.8183, -1.3959,  0.5713],
        [-1.6621,  2.8714,  0.5136,  1.9950, -0.6663]])

In [19]:
print(mll.W)
print(mll.b)

tensor([[-1.1258, -1.1524, -0.2506, -0.4339,  0.5988],
        [-1.5551, -0.3414,  1.8530,  0.4681, -0.1577],
        [ 1.4437,  0.2660,  1.3894,  1.5863,  0.9463],
        [-0.8437,  0.9318,  1.2590,  2.0050,  0.0537]])
tensor([[1., 1., 1., 1., 1.]])


# Building a Linear Layer with `torch.nn`
위에서 수행한 작업을 `pytorch`에서는 `torch.nn.Linear`라는 명령어로 쉽게 구현할 수 있습니다.
아래 예제를 보도록 하죠

In [2]:
import torch.nn as nn
#W = torch.randn(num_feature, 5)
L1 = nn.Linear(num_feature, 5)
Zll = L1(X_batch)
Zll

NameError: name 'num_feature' is not defined

In [31]:
L1.weight = nn.Parameter(W.T)
L1.bias.data.fill_(1.0)
Zll2 = L1(X_batch)
Zll2

tensor([[ 0.5305,  1.1570,  0.2352,  2.0200,  3.5516],
        [ 3.1858,  0.1941,  0.6759,  0.3632, -2.1292],
        [ 1.8391,  1.2113,  2.0038, -0.8603,  2.1906],
        [ 1.0724,  0.6143,  1.6056,  1.9838,  0.5979],
        [-0.1756,  1.5531,  1.2092,  1.7496,  5.7082],
        [ 0.8844,  1.7877,  2.0312, -1.8249,  1.9058],
        [ 1.0777,  1.1470,  1.6190,  0.6380,  2.7351],
        [ 1.0298,  1.1658,  2.0023,  0.5876,  3.0591],
        [ 1.9733,  1.0070,  0.8147, -1.1146, -2.2139],
        [-1.2479,  1.7962, -0.2809,  1.4853,  1.1872]],
       grad_fn=<AddmmBackward0>)

In [ ]:
Z